In [1]:
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')

In [2]:
import semiolog as slg

In [3]:
semiotic = slg.Cenematic("en_bnc_old_segments")

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at models/en_bnc_old_segments/paradigms/tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.
Using custom data configuration corpus-48c43d9b917d9e96
Reusing dataset text (/Users/Gianni/.cache/huggingface/datasets/text/corpus-48c43d9b917d9e96/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)
100%|██████████| 3/3 [00:00<00:00, 33.90it/s]
All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at models/en_bnc_old_segments/paradigms/tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further trainin

In [4]:
from random import randint

In [5]:
sent = semiotic.corpus.train["text"][randint(0,10000)]
sent = "ihave made my plans and imust stick tothem"
sent_tokenized = semiotic.syntagmatic.bert_tokenizer.decode(semiotic.syntagmatic.bert_tokenizer(sent)["input_ids"])
print(sent)
print(sent_tokenized)

ihave made my plans and imust stick tothem
ihave made my plans and imust stick tothem


In [6]:
sent_seq = semiotic(sent)

In [7]:
bla = sent_seq.chain.tokens[6]
bla.paradigm.entropy

8.467295

In [8]:

slg.util.df([list(p.paradigm.keys)[:20] for p in sent_seq.chain.tokens], keys = sent_tokenized.split())

,ihave,made,my,plans,and,imust,stick,tothem
0,ihavent,nothingtodowith,detailsofthe,advice,that,iwouldliketo,speak,assoonaspossible
1,ihave,forgotten,someofthe,questions,toensurethat,startedto,listen,atthesametime
2,ivenever,changed,thefollowing,responsibility,because,opportunityto,talk,together
3,theyhave,alotof,thenecessary,comments,eventhough,decidedto,comeback,tothem
4,iwouldhave,discussed,attentiontothe,recommendations,but,continueto,respond,theproblem
5,inever,tochange,alotof,information,and,continuedto,contribute,forawhile
6,sihave,toaccept,anumberof,responsibilities,otherwise,idliketo,goback,everything
7,ihadnt,todealwith,considerable,arrangements,tomakesure,plansto,write,onbehalfofthe
8,wehave,responsibilityfor,their,conversation,although,difficultto,explain,verymuch
9,someofthem,implicationsfor,particular,impression,aboutwhat,promisedto,refer,verylittle


In [10]:
slg.util.df([list(p.paradigm.top_freq)[:20] for p in sent_seq.chain.tokens], keys = sent_tokenized.split())

,i,have,made,my,plans,and,i,must,stick,to,them
0,people,actually,thefirst,their,information,that,people,actually,havebeen,against,information
1,hesaid,havebeen,information,information,business,because,children,also,information,through,that
2,ithink,hadbeen,oneofthe,these,children,something,something,still,something,outof,something
3,andthen,hasbeen,both,the,company,and,thatthe,hadbeen,say,without,this
4,children,willbe,another,oneofthe,work,andthen,information,better,think,to,another
5,information,wouldbe,something,those,ofthem,although,ithink,hasbeen,know,with,business
6,ofthem,also,without,some,people,but,that,willbe,thefirst,withthe,government
7,both,still,therewas,thefirst,government,thatthe,there,thefirst,oneofthe,between,people
8,actually,oneofthe,all,government,room,information,ofthem,withthe,play,into,them
9,thereare,thefirst,found,thatthe,action,what,government,havebeen,ask,tothe,thefirst


In [10]:
max_parad = 100 # max_non_zeroes
slg.util.plot_scatter_line(list(range(max_parad)),sent_seq.chain.tokens[0].paradigm.values,sent_seq.chain.tokens[0].label, add_trace=[(list(range(max_parad)), token.paradigm.values, token.label) for token in sent_seq.chain.tokens[1:]])

In [11]:
max_parad = 100 # max_non_zeroes
slg.util.plot_scatter_line(list(range(max_parad)),sorted(sent_seq.chain.tokens[0].paradigm.probs, reverse=True),sent_seq.chain.tokens[0].label, add_trace=[(list(range(max_parad)), sorted(token.paradigm.probs, reverse=True), token.label) for token in sent_seq.chain.tokens[1:]])

In [13]:
import numpy as np
mass = [t.paradigm.mass for t in sent_seq.chain.tokens]
entropy = [t.paradigm.entropy for t in sent_seq.chain.tokens]
slg.util.plot_scatter_line(
    [f"{i}_"+k for i,k in enumerate(sent_seq.chain.labels)],
    mass/np.linalg.norm(mass),
    "mass",
    add_trace=[
        (
            [f"{i}_"+k for i,k in enumerate(sent_seq.chain.labels)],
            entropy/np.linalg.norm(entropy),
            "entropy"
        )
    ]
)

K-Means Clustering

In [177]:
from sklearn.cluster import KMeans
import numpy as np
from scipy.sparse import csr_matrix
from tqdm import trange


In [175]:
cutoff = 1000

In [178]:
indexes_i = []
values_i = []

for _ in trange(100):
    sent = semiotic.corpus.train["text"][randint(0,10000)]
    sent_seq = semiotic(sent)
    indexes_i += [token.paradigm.ids[:cutoff] for token in sent_seq.chain.tokens]
    values_i += [token.paradigm.values[:cutoff] for token in sent_seq.chain.tokens]

indexes = np.array(indexes_i)
values = np.array(values_i)

100%|██████████| 100/100 [03:08<00:00,  1.88s/it]


In [182]:
indexes.shape

(2029, 1000)

In [179]:
length = indexes.max()+1
X = np.zeros((indexes.shape[0], length))

rows = np.indices((X.shape[0],)).reshape(-1, 1)
X[rows, indexes] = values
X = csr_matrix(X)

In [180]:
n_cluster = 20
kmeans = KMeans(n_clusters=n_cluster, random_state=0).fit(X)

In [181]:
cluster_labels = []
for cluster in kmeans.cluster_centers_:
    c_indeces = np.nonzero(cluster)[0]
    c_values = cluster[c_indeces]
    c_order = list(zip(c_values,c_indeces))
    cluster_labels.append([semiotic.vocab.decode[i] for v,i in sorted(c_order,reverse=True)[:20]])
slg.util.df(cluster_labels)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,and,made,and,the,services,side,it,ing,on,the,ofthe,only,for,new,5,time,and,is,consider,the
1,to,found,but,s,work,head,them,ed,is,that,inthe,in,in,in,4,one,s,are,understand,their
2,that,left,that,as,and,and,him,s,in,a,and,well,to,good,7,work,in,was,explain,new
3,they,had,as,p,information,back,this,and,as,their,the,now,with,great,3,position,a,were,enjoy,its
4,i,kept,when,l,resources,s,that,edby,a,this,onthe,so,from,high,2,action,as,and,use,other
5,who,brought,because,t,problems,water,one,ers,an,his,tothe,and,of,small,1,system,all,itis,avoid,all
6,but,seen,if,and,companies,door,us,edin,s,its,of,more,on,old,6,order,not,maybe,find,some
7,we,lost,so,f,data,line,you,sand,and,some,fromthe,again,by,the,9,place,is,as,meet,these
8,which,called,for,them,people,wall,her,es,i,any,in,all,and,free,8,day,the,willbe,recognise,our
9,or,put,where,d,policy,light,me,able,am,my,andthe,quite,as,open,13,question,i,canbe,keep,both


In [ ]:
kmeans.predict([[0, 0], [12, 3]])